<a href="https://colab.research.google.com/github/ByungjunKim/OpenAlexAPI/blob/main/OpenAlex_API_240326.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -U requests natsort tqdm

In [2]:
import pandas as pd
import requests
from tqdm.auto import tqdm
tqdm.pandas()
import json
import glob
import time
import re
from natsort import natsorted
import itertools
import matplotlib.pyplot as plt

In [3]:
# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.0.0 Safari/537.36",
#     "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
#     "Accept-Language": "en-US,en;q=0.9",
#     "Accept-Encoding": "gzip, deflate, br",
#     "Connection": "keep-alive",
#     "DNT": "1", # Do Not Track 요청 헤더 (사용자 추적 거부)
#     # 필요한 추가 헤더를 여기에 추가
# }

In [31]:
def request_until_success(url, timeout=25, delay=5):
    """
    Continuously makes a request to the specified URL with a timeout until the request is successful.

    Args:
    - url (str): The URL to request.
    - timeout (int): The timeout for the request in seconds.
    - delay (int): The delay between retries in seconds.

    Returns:
    - response: The successful response from the server.
    """
    attempt = 1
    while True:
        try:
            # print(f"Attempt {attempt}")
            # response = requests.get(url,headers=headers, timeout=timeout)
            response = requests.get(url, timeout=timeout)
            response.raise_for_status()  # Raises HTTPError for bad responses
            # print("Request successful")
            return response.json()
        except requests.Timeout as e:
            print(f"Request timed out: {e}")
        except requests.RequestException as e:
            print(f"Request failed: {e}")

        # print(f"Retrying in {delay} seconds...")
        time.sleep(delay)
        attempt += 1

In [7]:
email = 'kuntakim88@gmail.com'
cursor = '*'

### title and abstract

In [12]:
# url with a placeholder for cursor
example_url_with_cursor = f'https://api.openalex.org/works?filter=title_and_abstract.search:japan,publication_year:2000&per-page=200&cursor={cursor}&mailto={email}'

In [30]:
page_with_results = requests.get(example_url_with_cursor).json()

In [19]:
# metadata
page_with_results['meta']

{'count': 24671,
 'db_response_time_ms': 151,
 'page': None,
 'per_page': 200,
 'next_cursor': 'IlsyMDIuMzA2MjEsIDk0NjY4NDgwMDAwMCwgJ2h0dHBzOi8vb3BlbmFsZXgub3JnL1cyMDcwOTc0NDA2J10i',
 'groups_count': None}

In [20]:
# results
page_with_results['results'][0]

{'id': 'https://openalex.org/W2163379050',
 'doi': 'https://doi.org/10.1785/0119990114',
 'title': 'Minimum Magnitude of Completeness in Earthquake Catalogs: Examples from Alaska, the Western United States, and Japan',
 'display_name': 'Minimum Magnitude of Completeness in Earthquake Catalogs: Examples from Alaska, the Western United States, and Japan',
 'relevance_score': 628.1769,
 'publication_year': 2000,
 'publication_date': '2000-08-01',
 'ids': {'openalex': 'https://openalex.org/W2163379050',
  'doi': 'https://doi.org/10.1785/0119990114',
  'mag': '2163379050'},
 'language': 'en',
 'primary_location': {'is_oa': False,
  'landing_page_url': 'https://doi.org/10.1785/0119990114',
  'pdf_url': None,
  'source': {'id': 'https://openalex.org/S59098468',
   'display_name': 'Bulletin of the Seismological Society of America',
   'issn_l': '0037-1106',
   'issn': ['1943-3573', '0037-1106'],
   'is_oa': False,
   'is_in_doaj': False,
   'host_organization': 'https://openalex.org/P431031685

In [21]:
end = int(page_with_results['meta']['count']/200)+1
start = 0
print(start, end)

0 124


In [22]:
!mkdir data

In [34]:
for idx in tqdm(range(start,end)):
    # set cursor value and request page from OpenAlex
    num = 200
    # url = f'https://api.openalex.org/works?filter=primary_location.source.id:{jour}&per-page={num}&cursor={cursor}&mailto=kuntakim88@gmail.com'
    url = f'https://api.openalex.org/works?filter=title_and_abstract.search:japan,publication_year:2000&per-page={num}&cursor={cursor}&mailto={email}'
    # url = base_url.format(cursor)
    # page_with_results = requests.get(url).json()
    page_with_results = request_until_success(url)

    # json으로 저장
    with open(f'./data/japn_{idx}.json','w') as f:
        json.dump(page_with_results,f)
    # next page
    cursor = page_with_results['meta']['next_cursor']

  0%|          | 0/124 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [35]:
# json file list
file_list = natsorted(glob.glob('./data/*.json'))
print(len(file_list))
file_list

9


['./data/japn_0.json',
 './data/japn_1.json',
 './data/japn_2.json',
 './data/japn_3.json',
 './data/japn_4.json',
 './data/japn_5.json',
 './data/japn_6.json',
 './data/japn_7.json',
 './data/japn_8.json']

In [36]:
# 여러 json 파일을 읽어서 result만 하나의 리스트로 저장하기
openalex_json = []
for file in tqdm(file_list):
    with open(file,'r') as f:
        temp_json = json.load(f)['results']
    openalex_json.extend(temp_json)

  0%|          | 0/9 [00:00<?, ?it/s]

In [37]:
# pandas dataframe으로 변환
df = pd.DataFrame.from_dict(openalex_json)
df

,id,doi,title,display_name,relevance_score,publication_year,publication_date,ids,language,primary_location,...,grants,referenced_works_count,referenced_works,related_works,ngrams_url,abstract_inverted_index,cited_by_api_url,counts_by_year,updated_date,created_date
0,https://openalex.org/W2163379050,https://doi.org/10.1785/0119990114,Minimum Magnitude of Completeness in Earthquak...,Minimum Magnitude of Completeness in Earthquak...,628.176900,2000,2000-08-01,{'openalex': 'https://openalex.org/W2163379050...,en,"{'is_oa': False, 'landing_page_url': 'https://...",...,[],30,"[https://openalex.org/W27965200, https://opena...","[https://openalex.org/W2185688839, https://ope...",https://api.openalex.org/works/W2163379050/ngrams,"{'We': [0, 88, 128], 'mapped': [1], 'the': [2,...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2024, 'cited_by_count': 10}, {'year'...",2024-03-15T06:47:46.399822,2016-06-24
1,https://openalex.org/W2016839669,https://doi.org/10.1257/aer.90.1.30,Collateral Damage: Effects of the Japanese Ban...,Collateral Damage: Effects of the Japanese Ban...,618.174870,2000,2000-03-01,{'openalex': 'https://openalex.org/W2016839669...,en,"{'is_oa': False, 'landing_page_url': 'https://...",...,[],4,"[https://openalex.org/W1969057730, https://ope...","[https://openalex.org/W2992028967, https://ope...",https://api.openalex.org/works/W2016839669/ngrams,"{'The': [0], 'Japanese': [1, 33], 'banking': [...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2024, 'cited_by_count': 4}, {'year':...",2024-03-23T21:00:27.237120,2016-06-24
2,https://openalex.org/W2112376540,https://doi.org/10.1080/026999300379003,"Culture, Emotion, and Well-being: Good Feeling...","Culture, Emotion, and Well-being: Good Feeling...",593.068900,2000,2000-01-01,{'openalex': 'https://openalex.org/W2112376540...,en,"{'is_oa': False, 'landing_page_url': 'https://...",...,[],35,"[https://openalex.org/W434182623, https://open...","[https://openalex.org/W4311417398, https://ope...",https://api.openalex.org/works/W2112376540/ngrams,"{'We': [0], 'tested': [1], 'the': [2, 19, 30, ...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2024, 'cited_by_count': 4}, {'year':...",2024-03-19T20:23:44.574222,2016-06-24
3,https://openalex.org/W2066580768,https://doi.org/10.1016/s0261-5606(00)00006-1,Volatility spillover effects from Japan and th...,Volatility spillover effects from Japan and th...,532.183840,2000,2000-04-01,{'openalex': 'https://openalex.org/W2066580768...,en,"{'is_oa': False, 'landing_page_url': 'https://...",...,[],40,"[https://openalex.org/W41114315, https://opena...","[https://openalex.org/W3174705069, https://ope...",https://api.openalex.org/works/W2066580768/ngrams,"{'This': [0], 'study': [1], 'examines': [2], '...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2024, 'cited_by_count': 3}, {'year':...",2024-03-17T08:42:17.964580,2016-06-24
4,https://openalex.org/W2106429028,https://doi.org/10.1016/s0378-4266(99)00095-3,Efficiency and risk in Japanese banking,Efficiency and risk in Japanese banking,516.751160,2000,2000-10-01,{'openalex': 'https://openalex.org/W2106429028...,en,"{'is_oa': False, 'landing_page_url': 'https://...",...,[],36,"[https://openalex.org/W1973695731, https://ope...","[https://openalex.org/W2990638728, https://ope...",https://api.openalex.org/works/W2106429028/ngrams,"{'This': [0], 'paper': [1], 'investigates': [2...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2024, 'cited_by_count': 1}, {'year':...",2024-03-15T02:56:10.559151,2016-06-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,https://openalex.org/W2144157559,None,Therapy-related leukemia and myelodysplastic s...,Therapy-related leukemia and myelodysplastic s...,88.895030,2000,2000-02-01,{'openalex': 'https://openalex.org/W2144157559...,en,"{'is_oa': False, 'landing_page_url': 'https://...",...,[],0,[],"[https://openalex.org/W2013903939, https://ope...",https://

In [38]:
df = df.drop_duplicates(subset=['id']).reset_index(drop=True)

In [39]:
def index_to_text(abstract_inverted_index):
    abstract_index = {}
    for k, vlist in abstract_inverted_index.items():
        for v in vlist:
            abstract_index[v] = k
    abstract = ' '.join(abstract_index[k] for k in sorted(abstract_index.keys()))
    return abstract

In [40]:
# index to text
df['abstract'] = df[~pd.isna(df['abstract_inverted_index'])]['abstract_inverted_index'].progress_map(lambda x:index_to_text(x))
df['abstract']

  0%|          | 0/1584 [00:00<?, ?it/s]

0       We mapped the minimum magnitude of complete re...
1       The Japanese banking crisis provides a natural...
2       We tested the hypothesis that “good feelings”—...
3       This study examines the magnitude and changing...
4       This paper investigates the impact of risk and...
                              ...                        
1783                                                  NaN
1784    Black dot ringworm (BDR), caused by Trichophyt...
1785    Recent evidence suggests that an insertion/del...
1786    A systematic review was conducted to identify ...
1787    The carbon, nitrogen, phosphorus, and chloroph...
Name: abstract, Length: 1788, dtype: object

In [41]:
df.columns

Index(['id', 'doi', 'title', 'display_name', 'relevance_score',
       'publication_year', 'publication_date', 'ids', 'language',
       'primary_location', 'type', 'type_crossref', 'indexed_in',
       'open_access', 'authorships', 'countries_distinct_count',
       'institutions_distinct_count', 'corresponding_author_ids',
       'corresponding_institution_ids', 'apc_list', 'apc_paid', 'has_fulltext',
       'fulltext_origin', 'cited_by_count', 'cited_by_percentile_year',
       'biblio', 'is_retracted', 'is_paratext', 'primary_topic', 'topics',
       'keywords', 'concepts', 'mesh', 'locations_count', 'locations',
       'best_oa_location', 'sustainable_development_goals', 'grants',
       'referenced_works_count', 'referenced_works', 'related_works',
       'ngrams_url', 'abstract_inverted_index', 'cited_by_api_url',
       'counts_by_year', 'updated_date', 'created_date', 'abstract'],
      dtype='object')

##### topics

In [42]:
df['topics'][0]

[{'id': 'https://openalex.org/T10110',
  'display_name': 'Seismicity and Tectonic Plate Interactions',
  'score': 0.9999,
  'subfield': {'id': 'https://openalex.org/subfields/1908',
   'display_name': 'Geophysics'},
  'field': {'id': 'https://openalex.org/fields/19',
   'display_name': 'Earth and Planetary Sciences'},
  'domain': {'id': 'https://openalex.org/domains/3',
   'display_name': 'Physical Sciences'}},
 {'id': 'https://openalex.org/T10001',
  'display_name': 'Tectonic and Geochronological Evolution of Orogens',
  'score': 0.9943,
  'subfield': {'id': 'https://openalex.org/subfields/1908',
   'display_name': 'Geophysics'},
  'field': {'id': 'https://openalex.org/fields/19',
   'display_name': 'Earth and Planetary Sciences'},
  'domain': {'id': 'https://openalex.org/domains/3',
   'display_name': 'Physical Sciences'}},
 {'id': 'https://openalex.org/T10413',
  'display_name': "Mantle Dynamics and Earth's Structure",
  'score': 0.9847,
  'subfield': {'id': 'https://openalex.org/su

In [43]:
df['topics_name'] = df['topics'].dropna().progress_map(lambda x:[t['display_name'] for t in x])
df['subfield_name'] = df['topics'].dropna().progress_map(lambda x:[t['subfield']['display_name'] for t in x])
df['field_name'] = df['topics'].dropna().progress_map(lambda x:[t['field']['display_name'] for t in x])
df['domain_name'] = df['topics'].dropna().progress_map(lambda x:[t['domain']['display_name'] for t in x])

  0%|          | 0/1788 [00:00<?, ?it/s]

  0%|          | 0/1788 [00:00<?, ?it/s]

  0%|          | 0/1788 [00:00<?, ?it/s]

  0%|          | 0/1788 [00:00<?, ?it/s]

In [44]:
df['topics_name']

0       [Seismicity and Tectonic Plate Interactions, T...
1       [Banking and Finance in Economic Systems, Cons...
2       [Cultural Psychology and Values Research, Mora...
3       [Modeling and Forecasting Financial Volatility...
4       [Efficiency Analysis in Production and Resourc...
                              ...                        
1783    [Acute Myeloid Leukemia, Advancements in Lung ...
1784    [Epidemiology and Management of Fungal Infecti...
1785    [Renin-Angiotensin System in Cardiovascular Ph...
1786    [Diagnosis and Management of Alzheimer's Disea...
1787    [Marine Biogeochemistry and Ecosystem Dynamics...
Name: topics_name, Length: 1788, dtype: object

In [45]:
df.loc[df['topics_name'].str.len()>0,'topics_name'].map(lambda x:x[0]).value_counts()

Cultural Dynamics in Japan and East Asia                      76
Seismicity and Tectonic Plate Interactions                    39
Global Impact of Arboviral Diseases                           30
Helicobacter pylori Infection and Gastric Cancer              28
Tectonic and Geochronological Evolution of Orogens            23
                                                              ..
Economics of Health Care Systems and Policies                  1
Impact of Globalization on Economic Development and Policy     1
Theory of Constraints in Operations Management                 1
Corporate Tax Avoidance and Strategies                         1
Physical Metallurgy of Shape Memory Alloys                     1
Name: topics_name, Length: 734, dtype: int64